Ce TP est en deux parties


*   La première consiste à programmer quelques méthodes de classification multilabel
*   La seconde consiste à benchmarker des méthodes de classific aiton multilael sur un jeu de données, en utiisant un package dédié à la classification multilabels



# 1. Programmation de méthodes de classification multilabel

## Lien avec le drive

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## Importing required packages

In [2]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn import metrics
from sklearn.metrics import accuracy_score, make_scorer, f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.base import clone
from random import shuffle

## 0. Load the Yeast Dataset

In [3]:
yeast = pd.read_csv('yeast.csv')

In [4]:
X = yeast.iloc[:, 0:-14]

In [5]:
y = yeast.iloc[:,-14:]

In [6]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.7)

## Définition de métriques

In [ ]:
# Que calculent les métriques suivantes ?

In [50]:
# Custom accuracy score

def get_accuracy_score(y_pred,y_test):

    if 'numpy' not in str(type(y_pred)):
        y_pred = y_pred.to_numpy()

    if 'numpy' not in str(type(y_test)):
        y_test = y_test.to_numpy()


    assert(y_test.shape == y_pred.shape)

    if y_pred.shape[1] <= 5: #For a smaller number of labels, a ratio of half the labels being correct is good enough
        ratio = 0.5
    else:
        ratio = 0.7 #For a number of labels, at least 70% of the predicted labels must be correct

    acc_rows = []

    for i in range(len(y_test)):
        acc_rows.append(np.count_nonzero(y_test[i]==y_pred[i]))

    acc_rows = [1 if x/y_pred.shape[1] >= ratio else 0 for x in acc_rows] #1 if ratio of match in a row is greater than ratio, else 0
    return sum(acc_rows)/len(acc_rows)

In [46]:
### Distance de Hamming

def HammingDistance_score(y_pred,y_test):

    if 'numpy' not in str(type(y_pred)):
        y_pred = y_pred.to_numpy()

    if 'numpy' not in str(type(y_test)):
        y_test = y_test.to_numpy()

    acc_rows = []

    for i in range(len(y_test)):
      acc_rows.append(np.count_nonzero(y_test[i]!=y_pred[i]))

    return sum(acc_rows)/len(acc_rows)


## 1. Implement the Binary Relevance Algorithm

Implémentez la classe BinaryRelevanceClassifier suivante qui prend en paramètre un estimateur et va l'utiliser sur chacun des labels qe ce soit en apprentissage ou en inférence.
* Vous stockerez les modèles dans une liste *model_list*
* Il est inutile d'implémenter la méthode *predict_proba*

In [11]:
class BinaryRelevanceClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, base_model=LogisticRegression()):
        self.base_model = base_model #base model - by default logistic regression

    def fit(self, X, y):
        self.model_list_ = []
        self.y_columns = y.columns
        for column in y.columns:
            y_column = y[column]
            model = clone(self.base_model)
            model.fit(X, y_column)
            self.model_list_.append(model)
        return self


    def predict(self,X):
        y_pred = pd.DataFrame()
        for i, model in enumerate(self.model_list_):
            y_pred[self.y_columns[i]] = model.predict(X)
        

        return y_pred

    def predict_proba(self,X):
        ...
        return None

Le code de la cellule suivante doit fonctionner si la classe BinaryRelevance ci-dessus est ok.

In [41]:
binclf = BinaryRelevanceClassifier(base_model=SVC(C=0.01, kernel='sigmoid', probability=True))
binclf.fit(X_train,y_train)
y_pred = binclf.predict(X_test)

In [ ]:
display('Accuracy : ',get_accuracy_score(y_test,y_pred), 'Hamming Distance : ', HammingDistance_score(y_test,y_pred) )
display('F1 score : ',f1_score(y_test, y_pred, average='macro'))

#



'Accuracy : '

0.8553719008264463

'Hamming Distance : '

3.2355371900826446

'F1 score : '

0.12049772347410956

# 2. Implement the label set method

## Construction des labels

In [ ]:
# le code ci-dessous recupère les données y dans un tableau numpy.

import numpy as np

# Convert the pandas DataFrame 'y' to a NumPy array.
y_array = y.to_numpy()

# Print the resulting NumPy array.
print(y)

      Class1  Class2  Class3  Class4  Class5  Class6  Class7  Class8  Class9  \
0          0       0       0       0       0       0       1       1       0   
1          0       0       1       1       0       0       0       0       0   
2          0       1       1       0       0       0       0       0       0   
3          0       0       1       1       0       0       0       0       0   
4          0       0       1       1       1       1       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
2412       0       1       1       0       0       0       0       0       0   
2413       1       1       0       0       0       0       0       0       0   
2414       0       0       0       0       0       1       1       1       0   
2415       0       0       0       0       0       0       0       0       0   
2416       0       1       1       0       0       0       0       0       0   

      Class10  Class11  Class12  Class1

In [ ]:
y_unique, index_unique, reverse_unique = np.unique(y_array,return_index=True, return_inverse=True, axis=0)


print(reverse_unique)




[12 51 78 ... 23  0 78]


Les tableaux sont repectivement: les lignes uniques dans y array, l'index de ces ligne et le nombre de fois que chaque unique label apparait dans y array

Que contiennent les tableaux créés ci-dessus ?

Construisez les données X_trainLS, X_testLS, y_trainLS, y_testLSpour la méthode Label Set.

In [23]:
# Construct y_trainLS and y_testLS using reverse_unique
y_trainLS = reverse_unique[:len(y_train)]
y_testLS = reverse_unique[len(y_train):]

# X_trainLS and X_testLS are the same as X_train and X_test
X_trainLS = X_train
X_testLS = X_test

print("X_trainLS shape:", X_trainLS.shape)
print("X_testLS shape:", X_testLS.shape)
print("y_trainLS shape:", y_trainLS.shape)
print("y_testLS shape:", y_testLS.shape)

X_trainLS shape: (1691, 103)
X_testLS shape: (726, 103)
y_trainLS shape: (1691,)
y_testLS shape: (726,)


## Expériences

La fonction `reconstruit_y` ci-dessous reconstruit les prédictions du modèle appris avec la méthode Label Set au format attendu par les métriques de performance définies plus haut. Expliquez le fonctionnement de cette fonction.


La fonction va prendre en entrée les labels et les données y et va les transformer en un tableau de labels binaires ( ici 726 * 14)

In [24]:
def reconstruit_y(yLS,  LS ): # reconstruit les vecteurs y (vecteurs d'indicateurs de labels yLS) à partir des LS (les labelsets)
  y_temp = [LS[ yLS[i]] for i in range(len(yLS))]
  y_temp = np.concatenate( y_temp, axis=0 )
  y_temp = y_temp.reshape((yLS.shape[0],LS.shape[1]))
  return y_temp

Faites quelques expériences avec la méthode Label Set.

In [ ]:
# Test de la fonction reconstruit_y
y_testLS_reconstruit = reconstruit_y(y_testLS, y_unique)
print("y_testLS_reconstruit shape:", y_testLS_reconstruit.shape)
print("y_testLS_reconstruit:")
print(y_testLS_reconstruit)





y_testLS_reconstruit shape: (726, 14)
y_testLS_reconstruit:
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 0]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 0]
 [0 1 1 ... 1 1 0]]
[[0 0 0 ... 1 1 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 ...
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 0]]


In [59]:
display('Accuracy : ',get_accuracy_score(y_testLS_reconstruit ,y_pred), 'Hamming Distance : ', HammingDistance_score(y_testLS_reconstruit,y_pred) )
display('F1 score : ',f1_score(y_testLS_reconstruit, y_pred, average='macro'))

'Accuracy : '

0.8429752066115702

'Hamming Distance : '

3.239669421487603

'F1 score : '

0.12128291974839289

## Task 3: Implement the Classifier Chains Algorithm

Cpmplétez la classe suivante qui implémente les classifier Chains.


In [67]:
class ClassifierChains(BaseEstimator, ClassifierMixin):

    def __init__(self, base_model=LogisticRegression(), order = None, undersample=False):
        self.base_model = base_model # The base estimator
        self.order = order # Order of labels in which the labels are to be sent to the classifier

    def fit(self, X, y):
        X_cpy = X.copy()
        y_cpy = y.copy()
        self.order_shuffle = None
        if self.order is None:
            self.order = list(range(y.shape[1]))
            random.shuffle(self.order)
        self.order_shuffle = self.order

        self.model_list_ = []
        for i in self.order_shuffle:
            model = clone(self.base_model)
            model.fit(X_cpy, y_cpy.iloc[:, i])
            self.model_list_.append(model)
            X_cpy = np.hstack((X_cpy, y_cpy.iloc[:, i].values.reshape(-1, 1)))
        return self



    def predict(self, X):
        X_cpy = X.copy()
        y_pred = np.zeros((X.shape[0], len(self.order_shuffle)))
        for i, model in enumerate(self.model_list_):
            y_pred[:, self.order_shuffle[i]] = model.predict(X_cpy)
            X_cpy = np.hstack((X_cpy, y_pred[:, self.order_shuffle[i]].reshape(-1, 1)))
        return pd.DataFrame(y_pred, columns=self.order_shuffle)

    def predict_proba(self,X):
        #Not to be implemented

        return None
        

        

Faites des expériences avec la méthode Classifer Chain

In [68]:
classchn = ClassifierChains(base_model=SVC(C=0.01, kernel='sigmoid', probability=True))
classchn.fit(X_train,y_train)
y_pred2 = classchn.predict(X_test)

In [69]:
display('Accuracy : ',get_accuracy_score(y_test,y_pred2), 'Hamming Distance : ', HammingDistance_score(y_test,y_pred2) )
display('F1 score : ',f1_score(y_test, y_pred2, average='macro'))

'Accuracy : '

0.8553719008264463

'Hamming Distance : '

3.2355371900826446

'F1 score : '

0.12049772347410956

# 2. Utilisation du package [scikit-multilearn](http://scikit.ml/modelselection.html)

Installez ce package et utilisez le pour reproduire des expériences ci-dessus sur les mêmes données.

Vosu ferez des expériences avec BR, Classifier Chain mais et des estimateurs de votre choix mais aussi avec de sméthodes naturellement transformées comme les KNN (MLKNN en version multilabel)

A noter que le code du package se trouve [ici](https://github.com/scikit-multilearn/scikit-multilearn/tree/master/skmultilearn) et qu'un bug est idenfitié sur la méthode MLKNN à corriger selon [ces indications](https://stackoverflow.com/questions/74613688/typeerror-skmultilearn-error-with-multilabel-knn) en redéfinissant la classe MLKNN.

Vous pourrez chercher les meileurs hyperparamètres avec un code tel que celui-ci pour le cas des MLKNNs.


```
from sklearn.model_selection import GridSearchCV


parameters = {'k': range(1,20)}
score = 'f1_weighted'

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X, y)

classifier.best_params_, classifier.best_score_
```



In [1]:
from skmultilearn.dataset import load_dataset
from .mlknn import MLkNN


X_train, y_train, feature_names, label_names = load_dataset('emotions', 'train')
X_test, y_test, _, _ =load_dataset('emotions', 'test')

np.unique(y_train.rows).shape, np.unique(y_test.rows).shape


classifier = MLkNN(k=3)
prediction = classifier.fit(X_train, y_train).predict(X_test)

ImportError: attempted relative import with no known parent package

In [74]:
parameters = {'k': range(1,20)}
score = 'f1_weighted'

classifier = GridSearchCV(MLkNN(), parameters, scoring=score)
classifier.fit(X, y)

classifier.best_params_, classifier.best_score_


NameError: name 'MLkNN' is not defined